In [1]:
from datetime import datetime, timedelta

In [2]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('russian')

In [3]:
stemmer.stem('читать')

'чита'

In [4]:
with open('data/football') as inp:
    print(next(inp))
    i = 0
    wordbags = []
    for line in inp:
        i += 1
        print(i, '\r', end='')
        words = line.split('\t')[0].split()
        dttm = datetime.strptime(line[:-1].split('\t')[1], '%Y-%m-%d %H:%M:%S')
        if i > 1000000:
            break
        bag = []
        for word in words:
            sword = stemmer.stem(word.lower())
            if len(sword) > 2:
                bag.append(sword)
        if len(bag) > 9:
            continue
        wordbags.append((sorted(bag), dttm))

normal_query	datetime



In [5]:
len(wordbags)

992110

In [6]:
football_related = []
doc_freq = {}
for bag, dttm in wordbags:
    if 'футбол' in bag:
        football_related.append(bag)
    for term in bag:
        doc_freq[term] = doc_freq.get(term, 0) + 1

In [7]:
len(football_related)

13859

In [8]:
related_dict = {}
for bag in football_related:
    for term in bag:
        related_dict[term] = related_dict.get(term, 0) + 1

In [9]:
idf_dict = {}
for key in related_dict:
    idf_dict[key] = related_dict.get(key, 0) / doc_freq[key]

In [12]:
i = 0
correlated = {}
for key, value in sorted(idf_dict.items(), key=lambda item: item[1], reverse=True):
    if related_dict[key] > 20 and value > 0.01 and key != 'футбол':
        print(key, value)
        correlated[key] = value
        i += 1
        if i == 1000:
            break

турнирн 0.9313795568263045
куличк 0.89
чемпионат 0.8348116911443944
2022 0.6590909090909091
полуфина 0.6111111111111112
таблиц 0.592185128983308
мир 0.5862815884476534
сборн 0.559153998678123
офф 0.5234375
фина 0.4346793349168646
бомбардир 0.42857142857142855
трансляц 0.4030261348005502
соста 0.3436830835117773
чемпион 0.3154121863799283
матч 0.2911506417473542
выигра 0.27014218009478674
расписан 0.2611761387674517
вратар 0.25301204819277107
тайм 0.23577235772357724
2020 0.23
вчер 0.22115384615384615
2018 0.21999112202000157
слепак 0.20300751879699247
сыгра 0.1977401129943503
пле 0.195906432748538
проведен 0.1870748299319728
прям 0.18706244749369924
египт 0.1780821917808219
гимн 0.1693548387096774
команд 0.1674347158218126
результат 0.16689008042895442
ставк 0.15889830508474576
сетк 0.1511627906976744
эфир 0.14405986903648269
счет 0.1381294964028777
проход 0.13707165109034267
кем 0.1308411214953271
росс 0.1304214559386973
сейчас 0.13005272407732865
испан 0.1291743119266055
египет 0.119

In [62]:
min_dt, max_dt = datetime.now(), datetime.now() - timedelta(days=1800)
with open('data/football') as inp:
    print(next(inp))
    for line in inp:
        d = datetime.strptime(line[:-1].split('\t')[1], '%Y-%m-%d %H:%M:%S')
        if d < min_dt:
            min_dt = d
        if d > max_dt:
            max_dt = d
print(min_dt, max_dt)

normal_query	datetime

2018-06-14 00:00:00 2018-07-15 23:59:59


In [29]:
correlated

{'футбол': 1.0,
 'турнирн': 0.9340970682434097,
 'куличк': 0.925,
 'чемпионат': 0.8335878068883514,
 'офсайд': 0.6292134831460674,
 '2026': 0.6218487394957983,
 'таблиц': 0.6114451656986676,
 '2022': 0.5969868173258004,
 'мир': 0.5966909267345949,
 'сборн': 0.5593536357986327,
 'полуфина': 0.546,
 'полуфинал': 0.5277777777777778,
 'офф': 0.502594514455152,
 'транслир': 0.4845360824742268,
 'фина': 0.4666539996199886,
 'слепаков': 0.44505494505494503,
 'фаворит': 0.43103448275862066,
 'следующ': 0.4094955489614243,
 'жеребьевк': 0.40860215053763443,
 'трансляц': 0.4034997756554067,
 'подгрупп': 0.3867924528301887,
 'отборочн': 0.37943262411347517,
 'участв': 0.3785900783289817,
 'суперкубок': 0.3776223776223776,
 'чемпион': 0.3679411764705882,
 'церемон': 0.36079545454545453,
 'четвертьфинал': 0.34579439252336447,
 'призер': 0.34328358208955223,
 'соста': 0.3416287196865403,
 'четвертьфина': 0.3411764705882353,
 'финалист': 0.3387096774193548,
 'бомбардир': 0.3348115299334812,
 'уругв':

In [13]:
related_bags = []
for bag in wordbags:
    score = 0
    for term in bag[0]:
        if term in correlated:
            score += correlated[term]
        if score != 0:
            related_bags.append((bag[0], bag[1], score / len(bag[0])))

In [21]:
with open('data/refined', 'w') as f:
    for row in sorted(related_bags, key=lambda x: x[2], reverse=True)[:100000]

[(['2018', '2018', 'летн', 'нов', 'песн', 'сам', 'супер', 'хит', 'шикарн'],
  datetime.datetime(2018, 6, 27, 17, 6, 6),
  0.048886916004444794),
 (['2018', '2018', 'летн', 'нов', 'песн', 'сам', 'супер', 'хит', 'шикарн'],
  datetime.datetime(2018, 6, 27, 17, 6, 6),
  0.048886916004444794),
 (['2018', '2018', 'летн', 'нов', 'песн', 'сам', 'супер', 'хит', 'шикарн'],
  datetime.datetime(2018, 6, 27, 17, 6, 6),
  0.048886916004444794),
 (['2018', '2018', 'летн', 'нов', 'песн', 'сам', 'супер', 'хит', 'шикарн'],
  datetime.datetime(2018, 6, 27, 17, 6, 6),
  0.048886916004444794),
 (['2018', '2018', 'летн', 'нов', 'песн', 'сам', 'супер', 'хит', 'шикарн'],
  datetime.datetime(2018, 6, 27, 17, 6, 6),
  0.048886916004444794),
 (['2018', '2018', 'летн', 'нов', 'песн', 'сам', 'супер', 'хит', 'шикарн'],
  datetime.datetime(2018, 6, 27, 17, 6, 6),
  0.048886916004444794),
 (['2018', '2018', 'летн', 'нов', 'песн', 'сам', 'супер', 'хит', 'шикарн'],
  datetime.datetime(2018, 6, 27, 17, 6, 6),
  0.048886

In [45]:
groupby_related = {}
for bag in related_bags:
    groupby_related[(bag[0], bag[2])] = groupby_related.get((bag[0], bag[2]), 0) + 1 -- + score

TypeError: unhashable type: 'set'